# Details and FOS Calculation for the Supsension System

## Breaking Strength of Suspension Cables

We have 6 cables of varying lengths, of two distinct diameters: 13 mm and 19 mm. Refer to `./files/TIM SUSPENSION CABLES REV A SEPT26-2022.pdf` for details; we actually are going to use 19 mm diameter instead of the quoted 18 mm (for better values of breaking strength) and different turnbuckles (due to availability). 

We purchase our 6 cables with the attached Crosby spelters (compatible with the rest of the assembly), as well as [Green Pin EJ Turnbuckles G6315 1"$\times$ 6" (#SSGPGO2506)](http://www.greenpin.com/en/product/green-pinr-ej-turnbuckle#a_specifications) from [Nelson Wire Rope](https://www.nelsonwirerope.com/) ([catalog](https://indd.adobe.com/view/f5703249-a41b-4a9c-ab27-c0ab4a398243)). Nelson rates the cables to 45.209 and 20.4 US Tons respectively for 19 and 13 mm diameters (these are comparanle to the values Nate works with in his thesis).

The constrution of the cables is [CASAR Betalift](https://www.casar.de/Portals/0/Documents/Product-Specs/Betalift_02_2020.pdf?ver=DXyjJKf_BThaLCPJv28a1g%3d%3d), as with BLAST-TNG, which is non-rotation resistant, galvanized, lubricated, and rated to work within a large enough temperature range. In the linked catalog, for the construction we use (2160 $\text{N/mm}^2$), the cables for 13 mm and 19 mm diameters are rated to 20.39 and 43.91 US Tons respectively. With a abundance of caution, we are going to use these lower numbers in the following Factor of Safety (FOS) calculation. 

The Ultimate Tensile Strength is 5 times this quoted Rated Capacity.

## Breaking Strength of Turnbuckles

The [Green Pin catalog](https://www.greenpin.com/en/product/green-pinr-ej-turnbuckle#a_specifications) lists the working load limit (WLL) at 4.54 US Tons, at a safety factor of 5. So, the minimum breakng load (MBL) is 22.7 US Tons, which is higher than their corresponding 13 mm diameter cables. So, the limiting factor here has to be the cables. We still do the calculations below for completion though.

## CSBF Requirements

[CSBF's Structural Design Requirements](https://www.csbf.nasa.gov/documents/gondola/820-PG-8700.0.1%20Gondola%20Structural%20Design%20Requirements.pdf) for a suspension system is listed as:
> A load 8 times the weight of the payload applied vertically at the suspension point.

> For multiple-cable suspension systems, each cable must have an ultimate strength greater than 4 times the weight of the payload divided by the sine of the angle that the cable makes with horizontal ($\alpha \geq$ 30 degrees) in a normal flight configuration.

(note the *each* for the second requirement; so the 8G load is split between the cables (for suspension), while the 4G load is not)

Relevant expressions are in [Nate's thesis](https://repository.upenn.edu/edissertations/3147/) are 4.2-4.3 (though there's a typo in 4.3) and in Section 3.5 of the CSBF guideline.

$$F_{nG} = n \times M / sin \theta$$
$$FOS = F_{breaking} / F_{nG}$$

with $n$ being 4 and 8 for the current CSBF guideline.

We pick the values for angle with horizonatal $\theta$ that are *different* from James' [TIM ODRM presentation](https://docs.google.com/presentation/d/1bBOVMKZTYi_infR2a1tj-Xd4C2k9hJUL1wJpEhd7FtU/edit#slide=id.gf905fd1ac0_0_381), reconfirmed from the `October26-2022` SolidWorks assembly by Shubh.

| Suspension Element | Construction | $\theta$ (horizontal) | Rated Capacity |
| ----------- | ----------- | ----------- | ----------- | 
| Upper Front Cable | CASAR Betalift 13 mm | 48.176 $\degree$ | 20.39 US Tons |
| Lower Front Cable | CASAR Betalift 13 mm | 75.75 $\degree$ | 20.39 US Tons | 
| Rear Cable | CASAR Betalift 19 mm | 65.954 $\degree$ | 43.91 US Tons |
| Front Turnbuckle | Green Pin 1" $\times$ 6" | 75.75 $\degree$ | 22.7 US Tons | 
| Rear Turnbuckle | Green Pin 1" $\times$ 6" | 65.954 $\degree$ | 22.7 US Tons | 

We work off a nominal upper limit of payload mass of 7000 lbs.

In [24]:
import pandas as pd
import numpy as np

mass = 7000 # lbs

df = pd.DataFrame({'Suspension Element': ['Upper Front Cable', 'Lower Front Cable', 'Rear Cable', 'Front Turnbuckle', 'Rear Turnbuckle'],
    'Construction': ['CASAR Betalift 13 mm', 'CASAR Betalift 13 mm', 'CASAR Betalift 19 mm', 'Green Pin', 'Green Pin'],
    'theta': [48.176, 75.75, 65.954, 75.75, 65.954], 'Rated Capacity': [20.39, 20.39, 43.91, 22.7, 22.7]})

df['Ultimate  Tensile Strength'] = df['Rated Capacity'] * 5

df['Force (4G)'] = mass * 4 / np.sin(np.deg2rad(df['theta']))
df['Force (8G)'] = mass * 8 / np.sin(np.deg2rad(df['theta'])) / 4

df['FOS (4G)'] =  df['Ultimate  Tensile Strength'] * 2000 / df['Force (4G)']
df['FOS (8G)'] = df['Ultimate  Tensile Strength'] * 2000 / df['Force (8G)']

df

,Suspension Element,Construction,theta,Rated Capacity,Ultimate Tensile Strength,Force (4G),Force (8G),FOS (4G),FOS (8G)
0,Upper Front Cable,CASAR Betalift 13 mm,48.176,20.39,101.95,37573.970173,18786.985087,5.426629,10.853258
1,Lower Front Cable,CASAR Betalift 13 mm,75.750,20.39,101.95,28888.884702,14444.442351,7.058078,14.116156
2,Rear Cable,CASAR Betalift 19 mm,65.954,43.91,219.55,30660.785453,15330.392726,14.321225,28.642450
3,Front Turnbuckle,Green Pin,75.750,22.70,113.50,28888.884702,14444.442351,7.857693,15.715387
4,Rear Turnbuckle,Green Pin,65.954,22.70,113.50,30660.785453,15330.392726,7.403594,14.807188


In [5]:
48.176 + 41.824, 65.954 + 24.046

(90.0, 90.0)